# plot

> A module to plot the results of the benchmark

In [ ]:
#| default_exp plot

In [ ]:
#| export
from fasterbench.utils import *
import re, numbers
from typing import Mapping, Sequence, List, Dict, Any
import plotly.graph_objects as go

In [ ]:
#| export
_UNIT_RX = re.compile(r"([.\d]+)\s*([kKmMgGtT]?)[bB]?$")

#| export
_SCALE   = dict(K=1e3, M=1e6, G=1e9, T=1e12,
                k=1e3, m=1e6, g=1e9, t=1e12)

#| export
def _parse(x: Any) -> float:
    if isinstance(x, numbers.Number): return float(x)
    m = _UNIT_RX.match(str(x))
    if not m: raise ValueError(f"Cannot parse metric value: {x}")
    num, suf = m.groups(); return float(num) / _SCALE.get(suf, 1.0)


#| export
SPECS: Dict[str, tuple] = {
    "💾": (lambda d: d["size_disk_bytes"] / 1e6,          "Size : {:.1f} MiB", 500),
    "🧮": (lambda d: _parse(d["size_num_params"]) /1e6,   "Params: {:.2f} M",   50),
    "⏱️": (lambda d: d["speed_cpu_mean_ms"],              "Latency: {:.1f} ms", 2000),
    "🌍": (lambda d: _parse(d["energy_cpu_co2_eq_g"]) * 1e3,    "Emission: {:.3f} mgCO2e", 5),
    "🔋": (lambda d: d["energy_cpu_energy_wh"] * 1e3,         "Energy: {:.3f} mWh", 10),
}

#| export
def create_radar_plot(
    benchmark_results : Sequence[Mapping[str, Any]],
    model_names      : Sequence[str] | None = None,
    reference_max    : Mapping[str, float] | None = None,
):

    if model_names is None:
        model_names = [f"Model {i}" for i in range(len(benchmark_results))]

    # ---------- collect raw values -------------------------------------
    raw_matrix: Dict[str, List[float]] = {e: [] for e in SPECS}
    hover_mat : Dict[str, List[str]]   = {e: [] for e in SPECS}

    for res in benchmark_results:
        for emoji, (extract, fmt, _) in SPECS.items():
            val = extract(res)
            raw_matrix[emoji].append(val)
            hover_mat[emoji].append(fmt.format(val))

    # ---------- reference range (min=0) --------------------------------
    ref_max = {e: max(max(vals), default)    # default from SPECS
               for e, (_, _, default) in SPECS.items()
               for vals in [raw_matrix[e]]}
    if reference_max:
        ref_max.update(reference_max)

    # ---------- normalise & build traces -------------------------------
    labels = list(SPECS.keys()) + [list(SPECS.keys())[0]]   # close loop
    fig = go.Figure()
    ORANGE = "#FF8C00"

    for idx, name in enumerate(model_names):
        norm   = []
        hovers = []
        for e in SPECS:
            val = raw_matrix[e][idx]
            norm.append(val / ref_max[e] if ref_max[e] else 0)
            hovers.append(hover_mat[e][idx])
        norm  += norm[:1]; hovers += hovers[:1]

        fig.add_trace(
            go.Scatterpolar(
                r          = norm,
                theta      = labels,
                name       = name,
                hovertext  = hovers,
                hoverinfo  = "text",
                #line       = dict(color=ORANGE),
                fill       = "toself",
                #fillcolor  = "rgba(255,140,0,0.2)",
            )
        )

    # ---------- layout --------------------------------------------------
    fig.update_layout(
        polar=dict(
            radialaxis=dict(range=[0,1], visible=True,
                            showticklabels=False,
                            gridcolor="rgba(128,128,128,0.4)",
                            linecolor="rgba(128,128,128,0.4)"),
            angularaxis=dict(tickfont=dict(size=24),
                             gridcolor="rgba(128,128,128,0.4)"),
            bgcolor="rgba(0,0,0,0)",
        ),
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor ="rgba(0,0,0,0)",
        margin=dict(t=60, b=60, l=60, r=60),
        showlegend=len(model_names) > 1,
    )
    return fig
